In [ ]:
import os
import requests
from dotenv import load_dotenv
import clickhouse_connect
import itertools
import pandas as pd
import json
load_dotenv()
clickhouse_host = os.getenv("CLICKHOUSE_HOST")
username = os.getenv("CLICKHOUSE_USER")
password = os.getenv("CLICKHOUSE_PASSWORD")
github_token = os.getenv("GITHUB_TOKEN")

client = clickhouse_connect.get_client(host=clickhouse_host, port=8123, username=username, password=password)

headers = {
  "Authorization": f"token {github_token}"
}

## 获取全域 OpenRank > 50 的项目列表

In [ ]:

# 查询 github openrank>50 top 项目列表
def execute_query_top_openrank(created_at='2025-01-01'):
  sql_query_top_openrank = """
    SELECT
        repo_id,
        repo_name,
        ROUND(AVG(openrank)) AS avg_openrank_25
    FROM
        opensource.global_openrank
    WHERE
        platform = 'GitHub' AND
        created_at >= %s
    GROUP BY
        repo_id, repo_name
    HAVING
        avg_openrank_25 >= 30 and avg_openrank_25 < 50
    ORDER BY
        avg_openrank_25 DESC

  """
  formatted_query = sql_query_top_openrank % (f"'{created_at}'")
  results = client.query(formatted_query)
  return results


results_openrank = execute_query_top_openrank()
print(f"Found {len(results_openrank.result_rows)} repositories with high OpenRank scores")

# 提取项目名称到列表
repo_names = [row[1] for row in results_openrank.result_rows]
print(f"First 5 repositories: {repo_names[:5]}")


## 添加项目基本信息

In [ ]:
import requests  

# repo_names = pd.read_csv('landscape.csv')['repo_name'].tolist()
repo_names = []

repo_data = []  # Initialize empty list for repo data

def fetch_repo_info(repo_names, headers):
  github_repo_url = "https://api.github.com/repos/"
  openrank_repo_url = "https://oss.open-digger.cn/github/{repo_name}/openrank.json"
  for repo_name in repo_names:
    response = requests.get(github_repo_url + repo_name, headers=headers)
    if response.status_code == 200:
      data = response.json()
      repo_id = data['id']
      stars = data['stargazers_count'] 
      forks = data['forks_count']
      language = data['language']
      created_at = data['created_at'].split("T")[0]
      description = data['description']
      topics = ','.join(data.get("topics", [])) # 将topics列表转换为逗号分隔的字符串
      openrank_url = openrank_repo_url.format(repo_name=repo_name)
      openrank_response = requests.get(openrank_url)
      if openrank_response.status_code == 200:
        openrank_json = openrank_response.json()
        openrank = openrank_json.get("2025-07")
      else:
        openrank = None
      
      repo_data.append({
        'repo_id': repo_id,
        'repo_name': repo_name,
        'stars': stars,
        'forks': forks, 
        'openrank_25': round(openrank) if openrank else None,
        'language': language,
        'created_at': created_at,
        'description': description,
        'topics': topics
      })
    else:
      print(f"Failed to fetch data for {repo_name}")
  
  return repo_data

## 通过 OSS 获取 OpenRank Trend 并存储

In [ ]:
# 读取 repository_data.csv 并获取 OpenRank 趋势数据
import pandas as pd
import requests
import json
from time import sleep

repo_names = []


results = []

# 遍历每个仓库获取 OpenRank 数据
for repo_name in repo_names:
    url = f"https://oss.open-digger.cn/github/{repo_name}/openrank.json"
    
    try:
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            
            # 获取7月份的openrank值
            july_openrank = None
            if "2025-07" in data:
                july_openrank = round(data["2025-07"])
            
            # 提取2025年的月度数据
            trends_2025 = []
            for month in range(1, 13):
                month_key = f"2025-{month:02d}"
                if month_key in data:
                    trends_2025.append(round(data[month_key]))
            
            # 保存结果
            results.append({
                'repo_name': repo_name,
                'july_2025_openrank': july_openrank,
                'trends_2025': str(trends_2025)
            })
            
            print(f"成功获取 {repo_name} 的OpenRank数据 - 7月值: {july_openrank}, 2025年趋势: {len(trends_2025)}个月")
            
        else:
            print(f"获取 {repo_name} 数据失败: {response.status_code}")
            results.append({
                'repo_name': repo_name,
                'july_2025_openrank': None,
                'trends_2025': '[]'
            })
            
    except Exception as e:
        print(f"处理 {repo_name} 时发生错误: {str(e)}")
        results.append({
            'repo_name': repo_name,
            'july_2025_openrank': None,
            'trends_2025': '[]'
        })

# 创建DataFrame并保存数据
df_openrank = pd.DataFrame(results)
df_openrank.to_csv('repository_openrank_data.csv', index=False)
print(f"已完成所有{len(repo_names)}个仓库的OpenRank数据获取和保存")
print(f"数据已保存到 repository_openrank_data.csv")
